# Part 4: AI vs Human Review 비교

**SNU AI Psychology Workshop - February 2026**

---

## 학습 목표
1. **실제 피어리뷰** 데이터 수집하기
2. **간단한 리뷰 에이전트** 직접 구현하기
3. **AI vs Human 리뷰** 비교 분석

---

## 실제 리뷰 데이터 소스

| 사이트 | 분야 | 특징 |
|--------|------|------|
| **[OpenReview](https://openreview.net/)** | AI/ML | ICLR, NeurIPS 등 공개 리뷰 |
| **[PeerJ](https://peerj.com/)** | 생명과학 | 리뷰 히스토리 공개 |
| **[F1000Research](https://f1000research.com/)** | 다학제 | 공개 피어리뷰 |

> 이번 실습에서는 **OpenReview**에서 리뷰를 수집합니다.

---
## Part 1: Setup

In [ ]:
# Cell 1: 환경 설정
import os
import sys

# Colab 환경 감지
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    WORKSHOP_DIR = "/content/drive/MyDrive/aiworkshop_Feb2026/"
    os.makedirs(WORKSHOP_DIR, exist_ok=True)
    os.chdir(WORKSHOP_DIR)
    IN_COLAB = True
    print("🌐 Colab 환경에서 실행 중")
except ImportError:
    current_dir = os.getcwd()
    if current_dir.endswith('notebooks'):
        os.chdir('..')
    WORKSHOP_DIR = os.getcwd()
    IN_COLAB = False
    print("💻 로컬 환경에서 실행 중")

print(f"작업 폴더: {WORKSHOP_DIR}")

In [ ]:
# Cell 2: API Key 로딩
import os

if IN_COLAB:
    try:
        from google.colab import userdata
        GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    except:
        GEMINI_API_KEY = None
else:
    from dotenv import load_dotenv
    load_dotenv(override=True)
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

if GEMINI_API_KEY and not GEMINI_API_KEY.startswith('your_'):
    os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY
    print("✅ Gemini API Key 로드 완료")
else:
    print("❌ Gemini API Key가 필요합니다")
    print("   발급: https://aistudio.google.com/apikey")

---
## Part 2: 실제 리뷰 수집

### Step 1: OpenReview에서 리뷰 다운로드

**직접 수집 방법:**

1. **[OpenReview.net](https://openreview.net/)** 접속
2. 관심 있는 학회 선택 (예: ICLR 2024, NeurIPS 2023)
3. 논문 선택 → **Reviews** 탭 클릭
4. 리뷰 내용 복사 → 텍스트 파일로 저장

**저장 위치:** `input/human_reviews/`

**파일 형식 예시:**
```
input/human_reviews/
├── review_1.txt
├── review_2.txt
└── review_3.txt
```

---

### 리뷰 파일 형식

각 `.txt` 파일에 아래 형식으로 저장하세요:

```
=== 논문 제목 ===
[논문 제목 여기]

=== 점수 ===
Overall: [점수]
Soundness: [점수]
Presentation: [점수]
Contribution: [점수]

=== 리뷰 내용 ===
[리뷰 전체 내용]

=== Strengths ===
[강점들]

=== Weaknesses ===
[약점들]
```

> ⚠️ **최소 3개 이상의 리뷰를 수집하세요!**

In [ ]:
# Cell 3: 리뷰 폴더 생성 및 확인
import os

REVIEW_DIR = os.path.join(WORKSHOP_DIR, "input", "human_reviews")
os.makedirs(REVIEW_DIR, exist_ok=True)

# 기존 리뷰 파일 확인
review_files = [f for f in os.listdir(REVIEW_DIR) if f.endswith('.txt')]

if len(review_files) >= 3:
    print(f"✅ {len(review_files)}개의 리뷰 파일 발견!")
    for f in review_files:
        print(f"   📄 {f}")
elif len(review_files) > 0:
    print(f"⚠️ {len(review_files)}개의 리뷰 파일 발견 (최소 3개 권장)")
    for f in review_files:
        print(f"   📄 {f}")
else:
    print("❌ 리뷰 파일이 없습니다!")
    print(f"\n📁 리뷰 저장 위치: {REVIEW_DIR}")
    print("\n위 마크다운 셀의 안내에 따라 OpenReview에서 리뷰를 수집하세요.")

In [ ]:
# Cell 4: 리뷰 파일 파싱
import re

def parse_review_file(filepath):
    """리뷰 텍스트 파일 파싱"""
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()
    
    review = {
        'file': os.path.basename(filepath),
        'title': '',
        'scores': {},
        'review': '',
        'strengths': '',
        'weaknesses': ''
    }
    
    # 섹션 파싱
    sections = re.split(r'===\s*(.+?)\s*===', content)
    
    for i in range(1, len(sections), 2):
        section_name = sections[i].strip().lower()
        section_content = sections[i+1].strip() if i+1 < len(sections) else ''
        
        if '논문' in section_name or 'title' in section_name:
            review['title'] = section_content
        elif '점수' in section_name or 'score' in section_name:
            # 점수 파싱
            for line in section_content.split('\n'):
                if ':' in line:
                    key, val = line.split(':', 1)
                    review['scores'][key.strip().lower()] = val.strip()
        elif '리뷰' in section_name or 'review' in section_name:
            review['review'] = section_content
        elif 'strength' in section_name or '강점' in section_name:
            review['strengths'] = section_content
        elif 'weakness' in section_name or '약점' in section_name:
            review['weaknesses'] = section_content
    
    return review

# 모든 리뷰 파싱
human_reviews = []
for f in review_files:
    filepath = os.path.join(REVIEW_DIR, f)
    review = parse_review_file(filepath)
    human_reviews.append(review)
    print(f"📄 {f}: {review['title'][:50]}...")

print(f"\n✅ 총 {len(human_reviews)}개 리뷰 로드 완료")

---
## Part 3: 간단한 리뷰 에이전트 구현

노트북 3에서는 **agentic-paper-review** (9노드)를 사용했습니다.

이번에는 **직접 간단한 리뷰 에이전트**를 구현해봅니다.

### 구조
```
[논문 텍스트] → [프롬프트] → [LLM] → [리뷰 + 점수]
```

복잡한 워크플로우 없이 **단일 프롬프트**로 리뷰를 생성합니다.

In [ ]:
# Cell 5: 간단한 리뷰 에이전트 정의
import google.generativeai as genai

genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))

REVIEW_PROMPT = """당신은 학술 논문 리뷰어입니다. 아래 논문을 읽고 상세한 리뷰를 작성하세요.

## 논문 내용:
{paper_content}

## 리뷰 형식:

### 1. 요약 (Summary)
논문의 주요 내용을 2-3문장으로 요약하세요.

### 2. 강점 (Strengths)
- 강점 1
- 강점 2
- 강점 3

### 3. 약점 (Weaknesses)
- 약점 1
- 약점 2
- 약점 3

### 4. 점수 (Scores)
각 항목을 1-4점으로 평가하세요:
- Soundness (방법론 엄밀성): [1-4]
- Presentation (명확성): [1-4]
- Contribution (기여도): [1-4]
- Overall (종합): [1-10]

### 5. 최종 결정
Accept / Weak Accept / Weak Reject / Reject 중 하나를 선택하고 이유를 설명하세요.
"""

def simple_review_agent(paper_content, model_name="gemini-2.0-flash"):
    """간단한 리뷰 에이전트"""
    model = genai.GenerativeModel(model_name)
    
    prompt = REVIEW_PROMPT.format(paper_content=paper_content[:15000])  # 토큰 제한
    
    response = model.generate_content(prompt)
    
    return {
        'review': response.text,
        'model': model_name
    }

print("✅ simple_review_agent() 정의 완료")

In [ ]:
# Cell 6: 샘플 논문으로 AI 리뷰 생성

# 샘플 논문 로드
SAMPLE_PATH = os.path.join(WORKSHOP_DIR, "input", "sample_manuscript.md")

if os.path.exists(SAMPLE_PATH):
    with open(SAMPLE_PATH, 'r', encoding='utf-8') as f:
        sample_paper = f.read()
    
    print(f"📄 샘플 논문 로드: {len(sample_paper)} 문자")
    print("\n🔄 AI 리뷰 생성 중...")
    
    ai_review = simple_review_agent(sample_paper)
    
    print("\n" + "="*60)
    print("🤖 AI 리뷰 결과")
    print("="*60)
    print(ai_review['review'])
else:
    print("❌ 샘플 논문이 없습니다")
    print(f"   경로: {SAMPLE_PATH}")

---
## Part 4: AI vs Human 비교

수집한 Human 리뷰와 AI 리뷰를 비교합니다.

In [ ]:
# Cell 7: 비교 분석
from IPython.display import display, HTML

if human_reviews and 'ai_review' in dir():
    print("="*70)
    print("📊 AI vs Human 리뷰 비교")
    print("="*70)
    
    # Human 리뷰 요약
    print("\n👤 Human 리뷰 ({0}개):".format(len(human_reviews)))
    for i, hr in enumerate(human_reviews, 1):
        print(f"\n--- 리뷰 {i}: {hr['file']} ---")
        print(f"논문: {hr['title'][:50]}...")
        if hr['scores']:
            print(f"점수: {hr['scores']}")
        if hr['strengths']:
            print(f"강점: {hr['strengths'][:100]}...")
        if hr['weaknesses']:
            print(f"약점: {hr['weaknesses'][:100]}...")
    
    # AI 리뷰 요약
    print("\n" + "="*70)
    print("\n🤖 AI 리뷰 요약:")
    print(ai_review['review'][:1000] + "...")
    
else:
    print("⚠️ Human 리뷰와 AI 리뷰가 모두 필요합니다")
    if not human_reviews:
        print("   - Human 리뷰: Part 2 완료 필요")
    if 'ai_review' not in dir():
        print("   - AI 리뷰: Cell 6 실행 필요")

In [ ]:
# Cell 8: 노트북 3 결과와 비교
import json

# 노트북 3 결과 로드
notebook3_result_path = os.path.join(WORKSHOP_DIR, "outputs", "3_agent_result", "review.json")

notebook3_review = None
if os.path.exists(notebook3_result_path):
    with open(notebook3_result_path, 'r', encoding='utf-8') as f:
        notebook3_review = json.load(f)
    
    if notebook3_review.get('status') == 'completed' and notebook3_review.get('review'):
        print("✅ 노트북 3 결과 로드 완료")
    else:
        print("⚠️ 노트북 3 결과가 비어있거나 실패 상태입니다")
        notebook3_review = None
else:
    print("⚠️ 노트북 3 결과 파일이 없습니다")
    print("   노트북 3을 먼저 실행하세요")

# 3가지 비교
if notebook3_review and 'ai_review' in dir():
    print("\n" + "="*70)
    print("📊 3가지 리뷰 방식 비교")
    print("="*70)
    
    comparison_data = [
        ["방식", "복잡도", "특징"],
        ["Human Review", "N/A", "실제 전문가 리뷰"],
        ["Simple Agent (노트북 4)", "단일 프롬프트", "빠르고 간단"],
        ["Agentic Review (노트북 3)", "9노드 워크플로우", "웹검색, 리플렉션 포함"]
    ]
    
    for row in comparison_data:
        print(f"{row[0]:<25} | {row[1]:<15} | {row[2]}")

---
## Part 5: 결과 저장 및 제출

In [ ]:
# Cell 9: 결과 저장
import json
from datetime import datetime

output_dir = os.path.join(WORKSHOP_DIR, "outputs", "4_comparison_result")
os.makedirs(output_dir, exist_ok=True)

# 비교 결과 저장
comparison_result = {
    'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'human_reviews_count': len(human_reviews) if human_reviews else 0,
    'human_reviews': human_reviews if human_reviews else [],
    'ai_simple_review': ai_review if 'ai_review' in dir() else None,
    'ai_agentic_review': notebook3_review if notebook3_review else None
}

json_path = os.path.join(output_dir, "comparison.json")
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(comparison_result, f, ensure_ascii=False, indent=2)

print(f"💾 결과 저장: {json_path}")

In [ ]:
# Cell 10: 결과 제출 (Forms 연동은 추후 추가)

print("="*60)
print("📋 실습 완료!")
print("="*60)

print("\n✅ 완료한 작업:")
print(f"   1. Human 리뷰 수집: {len(human_reviews) if human_reviews else 0}개")
print(f"   2. Simple AI 리뷰 생성: {'완료' if 'ai_review' in dir() else '미완료'}")
print(f"   3. 노트북 3 결과 비교: {'완료' if notebook3_review else '미완료'}")

print("\n💡 토론 질문:")
print("   1. Human 리뷰와 AI 리뷰의 가장 큰 차이점은?")
print("   2. Simple Agent vs Agentic Agent - 어떤 차이가 있었나요?")
print("   3. AI 리뷰가 실제로 유용하려면 무엇이 필요할까요?")

---
## 정리

### 이번 노트북에서 배운 것

1. **실제 피어리뷰 데이터 수집** - OpenReview 활용
2. **간단한 리뷰 에이전트 구현** - 단일 프롬프트 방식
3. **AI vs Human 비교** - 강점/약점 분석

### 리뷰 에이전트 비교

| 방식 | 장점 | 단점 |
|------|------|------|
| **Human** | 전문성, 맥락 이해 | 시간 소요, 편향 가능 |
| **Simple Agent** | 빠름, 일관성 | 깊이 부족 |
| **Agentic Agent** | 웹검색, 리플렉션 | 복잡, API 비용 |

### 참고 자료

- OpenReview: https://openreview.net/
- PeerRead Dataset: https://github.com/allenai/PeerRead
- Google Gemini API: https://ai.google.dev/